In [1]:
import polars as pl
import os
from pathlib import Path

VGDB_PROJECT_DIR = Path(os.environ['VGDB_PROJECT_DIR'])

STAGING_DIR            = VGDB_PROJECT_DIR / "data/processed/staging"
ANALYTICS_DIR          = VGDB_PROJECT_DIR / "data/processed/analytics"

FACT_GAME_METRICS_PATH = ANALYTICS_DIR / "fact_game_metrics.parquet"
DIM_PLATFORM_PATH      = ANALYTICS_DIR / "dim_platform.parquet"

STG_GAME_IDENTITY_PATH = STAGING_DIR / "stg_game_identity.parquet"
STG_GAME_INFO_PATH     = STAGING_DIR / "stg_game_info.parquet"

In [2]:
# GROUP BY + HAVING: Which platforms have at least 5,000 games

platform_counts = (
    pl.read_parquet(FACT_GAME_METRICS_PATH)
    .group_by("platform_id")
    .agg(pl.len().alias("game_count"))
    .join(pl.read_parquet(DIM_PLATFORM_PATH), on="platform_id")
    .filter(pl.col("game_count") >= 5000)
    .sort("game_count", descending=True)
)

In [3]:
platform_counts

platform_id,game_count,platform_name
u32,u32,str
30,276430,"""PC"""
43,111638,"""Web"""
50,67568,"""iOS"""
51,61659,"""macOS"""
22,42827,"""Linux"""
2,33227,"""Android"""
36,5398,"""PlayStation 4"""


In [4]:
# LEFT JOIN: Keep all games, even if platform info is missing

games     = pl.read_parquet(STG_GAME_INFO_PATH)
platforms = pl.read_parquet(DIM_PLATFORM_PATH)

left_join = (
    games
    .select(["game_id", "name"])
    .join(pl.read_parquet(FACT_GAME_METRICS_PATH), on="game_id", how="left")
)

In [5]:
left_join.head()

game_id,name,platform_id,released_date,rating,metacritic_score,ratings_count,reviews_count,added_status_owned,added_status_playing
i64,str,u32,date,f64,i64,i64,i64,i64,i64
1,"""D/Generation HD""",30,2015-10-23,0.0,null,2,4,88,0
1,"""D/Generation HD""",51,2015-10-23,0.0,null,2,4,88,0
1,"""D/Generation HD""",48,2015-10-23,0.0,null,2,4,88,0
1,"""D/Generation HD""",36,2015-10-23,0.0,null,2,4,88,0
1,"""D/Generation HD""",29,2015-10-23,0.0,null,2,4,88,0


In [6]:
# FULL JOIN: Combine everything, even unmatched rows

full_join = (
    pl.read_parquet(STG_GAME_IDENTITY_PATH)
    .join(pl.read_parquet(STG_GAME_INFO_PATH), on="game_id", how="full")
)

In [7]:
full_join.filter(pl.any_horizontal(pl.all().is_null())).head()

game_id,slug,game_id_right,slug_right,name,metacritic_score,released_date,tba,updated_at,website,rating,rating_top,playtime_hours,achievements_count,ratings_count,suggestions_count,game_series_count,reviews_count,platforms,developers,genres,publishers,esrb_rating,added_status_yet,added_status_owned,added_status_beaten,added_status_toplay,added_status_dropped,added_status_playing
i64,str,i64,str,str,i64,date,bool,datetime[μs],str,f64,i64,f64,i64,i64,i64,i64,i64,list[str],list[str],list[str],list[str],str,i64,i64,i64,i64,i64,i64
153798,"""noahs-tasty-confusion""",153798,"""noahs-tasty-confusion""","""Noah's Tasty Confusion""",null,2018-07-21,false,2019-01-09 12:41:06,null,0.0,0,0.0,0,0,27,0,0,"[""PC""]","[""Micha Faw""]","[""Action""]",[],null,0,0,0,0,0,0
242527,"""star-wings-a-space-adventure-2""",242527,"""star-wings-a-space-adventure-2""","""Star Wings: A space adventure!""",null,2013-06-06,false,2019-10-22 14:14:30,"""http://www.arphix.com""",0.0,0,0.0,0,0,112,0,0,"[""macOS""]","[""Arphix""]","[""Action"", ""Adventure""]","[""Renan Lobo""]","""Everyone 10+""",0,0,0,0,0,0
639822,"""rugoe-of-desert""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
507324,"""executioners-of-the-sun""",507324,"""executioners-of-the-sun""","""Executioners of the Sun""",null,2020-10-18,false,2020-10-19 03:12:14,null,0.0,0,0.0,0,0,41,0,0,"[""PC""]","[""FieryChocobo""]","[""Strategy""]",[],null,0,0,0,0,0,0
324206,"""blackjack-pro-21-vegas-casino""",324206,"""blackjack-pro-21-vegas-casino""","""Blackjack Pro: 21 Vegas Casino""",null,2012-10-23,false,2019-10-22 14:31:21,null,0.0,0,0.0,0,0,113,0,0,"[""iOS""]","[""Avalinx""]","[""Card""]","[""Avalinx""]","""Teen""",0,0,0,0,0,0


In [8]:
# ROW_NUMBER: Top 3 games per platform by rating

top_games = (
    pl.read_parquet(FACT_GAME_METRICS_PATH)
    .join(pl.read_parquet(STG_GAME_INFO_PATH).select(["game_id", "name"]), on="game_id")
    .with_columns(
        pl.col("rating")
            .rank(method="ordinal", descending=True)
            .over("platform_id")
            .alias("row_number")
    )
    .filter(pl.col("row_number") <= 3)
    .sort(["platform_id", "row_number"])
)

In [12]:
top_games

game_id,platform_id,released_date,rating,metacritic_score,ratings_count,reviews_count,added_status_owned,added_status_playing,name,row_number
i64,u32,date,f64,i64,i64,i64,i64,i64,str,u32
34524,1,1994-11-01,4.5,null,8,8,5,0,"""SimTower""",1
52987,1,1995-04-15,4.45,null,22,22,9,0,"""Gex (1994)""",2
52884,1,1993-12-10,4.37,null,469,473,209,22,"""Doom""",3
229144,2,null,4.83,null,6,6,4,0,"""Hide Online""",1
1279,2,2015-12-09,4.75,null,7,8,5,5,"""Cataclysm: Dark Days Ahead""",2
…,…,…,…,…,…,…,…,…,…,…
1279,50,2015-12-09,4.75,null,7,8,5,5,"""Cataclysm: Dark Days Ahead""",2
57758,50,1994-03-18,4.71,null,6,7,5,0,"""Shin Megami Tensei II""",3
392459,51,2014-06-26,4.78,null,9,9,8,0,"""Shovel Knight: Shovel of Hope""",1


In [10]:
# LAG: release trend per platform

trend = (
    pl.read_parquet(FACT_GAME_METRICS_PATH)
    .with_columns(pl.col("released_date").dt.year().alias("year"))
    .group_by(["platform_id", "year"])
    .agg(pl.len().alias("games_released"))
    .sort(["platform_id", "year"])
    .with_columns(pl.col("games_released").shift(1).over("platform_id").alias("prev_year"))
)

In [11]:
trend.tail(10)

platform_id,year,games_released,prev_year
u32,i32,u32,u32
51,2013,617,415
51,2014,2164,617
51,2015,4622,2164
51,2016,6801,4622
51,2017,9111,6801
51,2018,9972,9111
51,2019,9680,9972
51,2020,15310,9680
51,2021,11,15310
